In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os

from src.dynamic_pricing_data_loader import cargar_y_preparar_datos

In [2]:
def Get_Data():
    # Obtener el directorio de trabajo actual (ruta principal del proyecto).
    ruta_principal = os.getcwd()

    # Construir la ruta al archivo de configuración "config/config.json".
    config_path = os.path.join(ruta_principal, "config", "config.json")

    # Llamar a la función externa que carga y realiza preprocesamiento inicial.
    Frame = cargar_y_preparar_datos(config_path, ruta_principal)

    # Seleccionar solo las columnas relevantes para el análisis.
    Frame = Frame[["FECHA_CORRIDA", "HORA_SALIDA_CORRIDA", "CLASE_SERVICIO", 'IVA_TARIFA_BASE_TRAMO',
    "PAX_SUBEN", "TARIFA_BASE_TRAMO",'FECHA_OPERACION', 'HORA_OPERACION','VENTA','DISPONIBILIDAD_TRAMO',
    'HORAS_ANTICIPACION','ORIGEN', 'DESTINO','TIPO_CLIENTE','NUM_ASIENTO'
                  ]].copy()

    return Frame

In [3]:
def Prepare_Data():
    df=Get_Data()
    df['FECHA_CORRIDA'] = pd.to_datetime(
        df['FECHA_CORRIDA']
    )
    
    df["HORA_SALIDA_CORRIDA"] = pd.to_datetime(
        df["HORA_SALIDA_CORRIDA"]
    ).dt.time
    
    df['TBT']= df['TARIFA_BASE_TRAMO']-df['IVA_TARIFA_BASE_TRAMO']
    df['%_dif_TBT_Venta']= (df['TBT']-df['VENTA'])/df['TBT']
    df['TIPO_CLASE'] = np.where(
        df['CLASE_SERVICIO'].astype(str).str.contains('DOS PISOS', case=False, na=False),
        'DOS',
        'UNO'
    )
    return df 

In [4]:
def Data4RedNeuronal():
    df=Prepare_Data()
    df_total= pd.DataFrame()
    df_total['Origen-Destino'] = df['ORIGEN'].astype(str) + '-' + df['DESTINO'].astype(str)
    df_total['DiaSemana_Corrida']=df['FECHA_CORRIDA'].dt.dayofweek
    df_total['Hora_Corrida']=df['FECHA_CORRIDA'].dt.hour
    df_total[['NUM_ASIENTO','HORAS_ANTICIPACION','%_dif_TBT_Venta']]=df[['NUM_ASIENTO','HORAS_ANTICIPACION','%_dif_TBT_Venta']].copy()
    df_total['Mes_Corrida']=df['FECHA_CORRIDA'].dt.month
    df_total['Buen_Dia'] = df['FECHA_CORRIDA'].dt.dayofweek.isin([4,5,6,0]).astype(int)
    df_total['Buena_Hora'] = df['FECHA_CORRIDA'].dt.hour.isin([23,17,18,19,20]).astype(int)
    df_total['Buen_Mes'] = df['FECHA_CORRIDA'].dt.month.isin([3,4,5,6]).astype(int)
    df_total['Buen_Asiento'] = df['NUM_ASIENTO'].isin([1,2,3,4,5,6,7,8,9,10]).astype(int)
    # Crea un nuevo DataFrame con las variables dummy (codificación one-hot)
    df_dummies = pd.get_dummies(
        df['TIPO_CLIENTE'],
        prefix='TIPO_CLIENTE', # Prefijo para las nuevas columnas (ej: TIPO_CLIENTE_A)
        drop_first=False        # Elimina la primera categoría para evitar multicolinealidad
    ).astype(int)
    
    # Crea un nuevo DataFrame con las variables dummy (codificación one-hot)
    df_dummies1 = pd.get_dummies(
        df['TIPO_CLASE'],
        prefix='PISO', 
        drop_first=False
    ).astype(int)
    
    # Une las nuevas columnas dummy al DataFrame original
    df_total = pd.concat([df_total, df_dummies,df_dummies1], axis=1)
    df_total['VENTA']=df['VENTA'].copy()

    return df_total

In [5]:
FrameN=Data4RedNeuronal()

Memoria usada antes: 621.61 MB
Memoria usada después: 120.85 MB
Reducción: 80.6%


In [6]:
FrameN.head()

,Origen-Destino,DiaSemana_Corrida,Hora_Corrida,NUM_ASIENTO,HORAS_ANTICIPACION,%_dif_TBT_Venta,Mes_Corrida,Buen_Dia,Buena_Hora,Buen_Mes,Buen_Asiento,TIPO_CLIENTE_AGENCIA,TIPO_CLIENTE_CALL CENTER,TIPO_CLIENTE_INTERNET,TIPO_CLIENTE_NULL,TIPO_CLIENTE_TAQUILLA,PISO_DOS,PISO_UNO,VENTA
0,MEXN-GDLJ,6,0,15,8.0,-0.080864,1,1,0,0,0,0,0,1,0,0,1,0,888.359985
1,MEXN-GDLJ,1,0,28,59.0,-0.071422,1,0,0,0,0,0,0,1,0,0,1,0,880.599976
2,MEXN-GDLJ,1,0,28,59.0,-0.071422,1,0,0,0,0,0,0,1,0,0,1,0,880.599976
3,MEXN-GDLJ,1,0,29,59.0,-0.071422,1,0,0,0,0,0,0,1,0,0,1,0,880.599976
4,MEXN-GDLJ,1,0,29,59.0,-0.071422,1,0,0,0,0,0,0,1,0,0,1,0,880.599976


In [7]:
FrameN.to_excel('PruebaData4RedN.xlsx')